In [1]:
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool
from crewai_tools import tool, BaseTool

import os
import requests

In [2]:
os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'

os.environ["GOOGLE_CSE_ID"] = ""
os.environ["GOOGLE_API_KEY"] = ""

In [3]:
llm = ChatOpenAI(
    model = "crewai-llama2",
    base_url = "http://localhost:11434/v1")



In [31]:
# Tools

human_tool = load_tools(["human"])

# Google Search

search = GoogleSearchAPIWrapper()

google_tool = Tool(
    name = "google_search",
    description = "Search Google for matching images. ",
    func = search.run,
)

# Image Creation Tool
image_tool = load_tools(["dalle-image-generator"])

@tool("image_download_tool")
def image_download_tool(image_url: str, image_name: str) -> str:
    """Give the url of the image and the name the image should download as with the file ending."""

    response = requests.get(image_url)
    with open(f"{image_name}", "wb") as file:
    # Write the content of the response (image) to the file
        file.write(response.content)

    return


# Works as a way to make a tool but isn't compatible with load_tools

# def image_download(image_url: str, image_name: str) -> str:
#     response = requests.get(image_url)
#     with open(f"{image_name}", "wb") as file:
#     # Write the content of the response (image) to the file
#         file.write(response.content)

#     return

# image_download_tool2 = Tool(
#     name = "Image_Download_Tool",
#     func = image_download,
#     description = "Give the url of the image and the name the image should download as with the file ending."
# )


In [34]:
# Agents

# Creating a senior researcher agent with memory and verbose mode
manager = Agent(
  name = "Manager",
  #llm = llm,
  role='Senior_Manager',
  goal = (
    'Over see the creation of {topic}. '
    'Plan how to create {topic}. '
    'Organize the tasks that must be completed. '
    'Delegate these tasks to co-workers. '
  ),
  verbose = True,
  memory = True,
  backstory = (
    "Dedicated manager who organizes tasks and delegates them effectively. "
  ),
  allow_delegation = True,
  #tools = human_tool
)

programmer = Agent(
  name = "Programmer",
  #llm = llm,
  role='Senior_Coder',
  goal= (
    'Write perfect python code to create {topic}. '
    'Resolve any bugs or errors in the code. '
    "Do not finalize your answer until the reviewer informs you the game is complete. "
  ),
  verbose=True,
  memory=True,
  backstory=(
    "You create and write perfect python code. "
  ),
  #tools=[search_tool],
  allow_delegation = True
)

# Creating a writer agent with custom tools and delegation capability
tester =Agent(
  name = "Tester",
  #llm = llm,
  role='Tester',
  goal= (
    'Give feedback on {topic}, and ensure that the programmer knows of any buggs or errors. '
  ),
  verbose=True,
  memory=True,
  backstory=(
    "Dedicated game tester. "
  ),
  tools= human_tool,
  allow_delegation = True
)

reviewer = Agent(
    name = "Reviwer",
    role = 'Reviewer',
    goal = (
        "Ensure your coworkers continue working until the human is satisfied. "
        "Ensure that all human feedback has been implemented and the code works as intended. "
    ),
    verbose = True,
    memory = True,
    backstory = (
        "Dedicated to your job you love the human and will make sure your coworkers do their job. "
    ),
    #tools = human_tool,
    allow_delegation = True
)

compiler = Agent(
    name = "Senior_Compiler",
    role = "Code Compiler",
    goal =(
        'You write perfect python code. '
        'You compile code and updates to that code into a final code file.'
    ),
    verbose = True,
    memory = True,
    backstory = (
        "You love to read and write code and make sure that all updates are included. "
    ),
    allow_delegation = True
)
# # Creating artist agent

artist = Agent(
  name = "Artist",
  #llm = llm,
  role = "Artist",
  goal = (
    'Search images'
  ),
  verbose = True,
  memory = True,
  backstory = (
    "Is an artist who always finds the best image. "
  ),
  tools = image_tool,
  allow_delegation = True
)

downloader = Agent(
    name = "Downloader",
    role = "Downloader",
    goal = (
        'Download images created by the Artist to be used by the programmer. '
    ),
    verbose = True,
    memory = True,
    backstory = (
        "You love downloading images. "
    ),
    tools = [image_download_tool],
    allow_delegation = False
)

In [26]:
# Tasks

# Manager Task
manager_task = Task(
  description = (
    "Understand the task given. "
    "Plan the exection of the task, and delegate jobs to co-workers. "
  ),
  expected_output = (
    "A python program that runs {topic}. "
    "Do not finalize your answer until the human says the game is complete. "
    #"Ask the human for feed back on the game before finalizing your answer. "
    "Do not finalize your answer until the reviewer recieves confirmation from the human that the game is complete. "
  ),
  agent = manager,
  #output_file = "game.py",
  human_input = False,
  async_execution = False
)

# Programmer task
programmer_task = Task(
  description=(
    "Write a single file with perfect python code to create {topic}."
    "Take feedback from the tester and reviewer and implement any changes requested. "
    "Do not finalize your answer until the reviewer has recieved confirmation the game is complete. "
    "Make sure to use the art from the artist in the code for {topic} "
  ),
  expected_output=(
    "A single executable file that contains the code for {topic}. "
    "The executive file should only contain the code for {topic}. "
  ),
  #tools=[search_tool],
  agent= programmer,
  output_file = "game_code.py",
  human_input = False,
  async_execution = False
)

# Writing task with language model configuration
tester_task = Task(
  description=(
    "Test the game created by the programmer and report feedback including any bugs and errors. "
    "Give feedback to your co-workers on the results. "
    "Once you can find no more bugs or erros check with the human for feedback before finalizing your answer. "
    "Give the human feedback to your co-workers. "
    "Continue to request feedback from the human then give this feedback to the programmer until the human confirms the game is finished. "
  ),
  expected_output = 'A report to the programmer indicating any bugs or errors in the code.',
  #tools = human_tool,
  agent = tester,
  output_file = 'errors.md' , # Example of output customization
  human_input = True,
  async_execution = False
)

reviewer_task = Task(
    description = (
        #"Make sure all human feedback has been implemented. "
        #"Make sure the code works as intended. "
        "Request confirmation from the human on whether the game is complete. "
        "If the game is not complete tell the manager to implement any neccesarry changes. "
        "If the game is complete tell the programmer they may finalize their answer. "
    ),
    expected_output = 'A report to your co-workers indicating whether all the human feedback has been implemented. ',
    #tools = human_tool
    agent = reviewer,
    output_file = "review.md",
    human_input = True,
    async_execution = False
)

compiler_task = Task(
    description = (
        "Take the code written by the programmer and add code to implement the human feedback. "
    ),
    expected_output = 'Your Final answer must be the full python code with all buggs and errors fixed and all human feedback implemented. The final answer must only be the python code and nothing else. ',
    agent = compiler,
    output_file = "final_game_code.py",
    async_execution = False
)

# Artist task
artist_task = Task(
  description = (
    #"Find and download pictures. "
    "Generate images to be used in {topic}. "
    "Make sure these images are used in the game. "
  ),
  expected_output = "Generated Images to be used in {topic}",
  tools = image_tool,
  agent = artist,
  async_execution = False,
  output_file = 'image.md'
)

# Downloader task
downloader_task = Task(
  description = (
    #"Find and download pictures. "
    "Download the images into files to be used by the programmer. "
    "Make sure to input a good image name that the programmer will use to access the image. "
    "Make sure these images are used in the game. "
  ),
  expected_output = "Generated Images to be used in {topic}",
  tools = [image_download_tool],
  agent = downloader,
  async_execution = False,
  output_file = 'downloader.md'
)

In [7]:
# Crew Creation

# Forming the tech-focused crew with some enhanced configurations
game_crew = Crew(
  agents = [manager, programmer, tester, compiler, reviewer],
  tasks = [manager_task, programmer_task, tester_task, compiler_task, reviewer_task],
  #agents = [programmer],
  #tasks = [programmer_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  #manager_llm = ChatOpenAI(model = 'gpt-3.5-turbo'),
  verbose = False,
  memory=True,
  cache=True,
  max_rpm=100,
  share_crew=True,
  step_callback = print("STEP ################################################"),
)

STEP ################################################


In [27]:
# Forming the tech-focused crew with some enhanced configurations
art_crew = Crew(
  agents = [programmer, artist, downloader, tester],
  tasks = [programmer_task, artist_task, downloader_task, tester_task],
  #agents = [programmer],
  #tasks = [programmer_task],
  process=Process.sequential,  # Optional: Sequential task execution is default
  #manager_llm = ChatOpenAI(model = 'gpt-3.5-turbo'),
  verbose = True,
  memory=True,
  cache=True,
  max_rpm=100,
  share_crew=True,
  step_callback = print("STEP ################################################"),
)

2024-06-09 11:04:52,350 - 125815334197056 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


STEP ################################################


In [29]:
if os.path.exists('game_code.py'):
    os.remove("game_code.py")
if os.path.exists('final_game_code.py'):
    os.remove("final_game_code.py")
if os.path.exists('errors.md'):
    os.remove("errors.md")
if os.path.exists('review.md'):
    os.remove("review.md")
if os.path.exists('human_feedback.md'):
    os.remove("human_feedback.md")
if os.path.exists('image.md'):
    os.remove("image.md")
    
result = art_crew.kickoff(inputs={'topic': 'The game pong with a beach as the background'})

print(result)

 [DEBUG]: == Working Agent: Senior_Coder
 [INFO]: == Starting Task: Write a single file with perfect python code to create The game pong with a beach as the background.Take feedback from the tester and reviewer and implement any changes requested. Do not finalize your answer until the reviewer has recieved confirmation the game is complete. Make sure to use the art from the artist in the code for The game pong with a beach as the background 


> Entering new CrewAgentExecutor chain...
I need to delegate the task of creating the art for the beach background to the Artist.

Action: Delegate work to co-worker
Action Input: {"task": "Create beach background art for the game pong", "context": "I need a beautiful beach background image for the game pong"} 

I encountered an error while trying to use the tool. This was the error: 'NoneType' object has no attribute 'startswith'.
 Tool Delegate work to co-worker accepts these inputs: Delegate work to co-worker(task: str, context: str, coworker:

2024-06-09 11:06:44,341 - 125815334197056 - sqlite.py-sqlite:284 - WARNING: Insert of existing embedding ID: default-app-id--658c87f68d53e6468a94e2d0ddca87d74c3496b1e72d8e3503417e0054e6036e


Thought: I need to ask a question to the Artist regarding the issue encountered while delegating the task of creating the beach background art.

Action: Ask question to co-worker
Action Input: {"question": "Can you provide me with the beach background art for the game pong?", "context": "Encountered an error while trying to delegate the task of creating the beach background art"} 

I encountered an error while trying to use the tool. This was the error: 'NoneType' object has no attribute 'startswith'.
 Tool Ask question to co-worker accepts these inputs: Ask question to co-worker(question: str, context: str, coworker: Optional[str] = None, **kwargs) - Ask a specific question to one of the following co-workers: [Artist, Downloader, Tester]
The input to this tool should be the co-worker, the question you have for them, and ALL necessary context to ask the question properly, they know nothing about the question, so share absolute everything you know, don't reference things but instead exp